## API Interaction - Pull Vendor Model
In this example we log into a remote teamworkcloud instance and retrieve all literal reals descriptions and values.

Steps:
1. Login to the TWC API Server
2. Retrieve a list of the workspaces (called categories in the TWC GUI)
3. Select the workspace to retrieve a list of projects
4. Retrieve a list project from the defined workspace
5. Select a project to retrieve all elements (in memory json file)
6. Loop though the json file and retrieve the index of any Literal Real (['ldp:DirectContainer', 'uml:LiteralReal'])
7. Convert the list to a Description:Value dataframe
8. Write the dataframe to a file

In [1]:
# A little set up
import json
import requests # performs the curl function in python
import ipywidgets as widgets
from ipywidgets import Dropdown
from IPython.display import display
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import time
from anytree import Node, RenderTree

In [2]:
# Lets get a list of workspaces from the teamworkcloud server
# API --> https://18.205.77.131:8111/osmc/swagger/?url=https%3A%2F%2F18.205.77.131%3A8111%2Fosmc%2Fmanual
#serverIp = '18.205.77.131' # Avian server
serverIp = '10.107.1.92' # ADT MA server
serverPort = '8111'
call = '/osmc/workspaces?includeBody=True'
# authId = "amRlaGFydDpqa2QyMjE0MDA=" # avian id
authId = "amRlaGFydDpSemhlZ3hoQ1RoWmY=" # ADT MA ID
url = f'https://{serverIp}:{serverPort}{call}'
headers={"accept":"application/ld+json","authorization":f"Basic {authId}"}
resp_ws = requests.get(url,headers=headers, verify=False) # turn of verification here since our server is not super secure
workspaces = resp_ws.json() # Convert the response content to a json format
#print(json.dumps(workspaces, indent=4))

In [3]:
# lets create a combobox to list the avalible workspaces
# Build arrays of the items
workspaceIds = {}
workspaceNames = {}

# Lets build a list of workspaces for selection
for i in range(len(workspaces["ldp:contains"])):
    workspaceIds[i] = workspaces["ldp:contains"][i][0]['@id']
    workspaceNames[i] = workspaces["ldp:contains"][i][1]["dcterms:title"]

# Fuction to monitor change of dropdown
def dropdown_eventhandler(change):
    print(change.new) # Write the selected item to the log

# Now create a dropdown list of the avalible workspaces
ws = widgets.Dropdown(options = workspaceNames.values(), description = 'Workspaces:')
ws.observe(dropdown_eventhandler, names='value')

# Displya the combobox
display(ws)

Dropdown(description='Workspaces:', options=('ADT_Projects', 'Test_Projects', 'Project_Templates', 'Uncategori…

In [4]:
# Lets match up the id from the selected workspace (poor implementation)
wsIndex = list(filter(lambda x: workspaceNames[x] == ws.value, range(len(workspaceNames))))
workspaceId = workspaceIds[wsIndex[0]]
workspaceId = "a2bf1c2f-03fb-4cbe-bb3d-4edce160916f" # hard set Test_Projects

In [5]:
# Now lets list the models that are withing the selected workspace and place them in a dropbox
# Ok... lets continue and list all of the the projects by project UID in this workspace
call = f'/osmc/workspaces/{workspaceId}/resources'
url = f'https://{serverIp}:{serverPort}{call}'
resp_projects = requests.get(url,headers=headers, verify=False) # turn of verification here since our server is not super secure
projectsList = resp_projects.json()
projectsUidList = projectsList[1]['kerml:resources'] # Let just extract the UIDs for each project
projectsUidList

[{'@id': '050dbf5b-f690-4494-81ee-18740b88d86c'},
 {'@id': '4eb3de65-53f4-4d59-aca2-62de8351e81b'},
 {'@id': '5cb74688-9804-44b1-badb-4f09c5cb861e'},
 {'@id': 'ef9e7f30-2fc1-4677-a170-ec7035ed6b0e'},
 {'@id': 'b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4'}]

## So now we can see all of the projects contained in the workspace

In [6]:
# Lets loop throug the projects and create a dictionary of the resource (or model) details
projectsData = {}
for i in range(len(projectsUidList)):
    resourceId = projectsUidList[i]['@id'] # select the values for each id in the projectList
    call = f'/osmc/workspaces/{workspaceId}/resources/{resourceId}'
    url = f'https://{serverIp}:{serverPort}{call}'
    resp_projects = requests.get(url,headers=headers, verify=False) # turn of verification here since our server is not super secure
    #projectsData = json.dumps(resp_projects.json(), indent=4)
    projectsData[i] = resp_projects.json()
#print(json.dumps(projectsData, indent=4))

In [7]:
# lets create a combobox to list the avalible projects (models) in this workspace
# Build arrays of the items
projectIds = {}
projectNames = {}

# Lets build a list of workspaces for selection
for i in range(len(projectsData)):
    projectIds[i] = projectsData[i]['@base'].split("/")[7]
    projectNames[i] = projectsData[i]['metadata']['name'].split(".")[0]

# Now create a dropdown list of the avalible projects
prj = widgets.Dropdown(options = projectNames.values(), description = 'Projects:')
prj.observe(dropdown_eventhandler, names='value')

# Displya the combobox
display(prj)

Dropdown(description='Projects:', options=('ADT_Model2', 'TestVerdictKind', 'VendorA', 'MacroEngine_Profile', …

In [8]:
# Lets match up the id from the selected workspace (poor implementation)
prjIndex = list(filter(lambda x: projectNames[x] == prj.value, range(len(projectNames))))
projectId = projectIds[prjIndex[0]]
projectId = "5cb74688-9804-44b1-badb-4f09c5cb861e" # hard set VendorA

In [9]:
# Ok so here is the wierd part... there is not direct way to extract the elements of a project (or model)
# So we have to perform a 'diff' between the elements of the initial commit and the latest revision. 

# So lets get the latest revision number (or max revision number)
# curl -X GET "https://18.205.77.131:8111/osmc/workspaces/bb95d8f4-fae4-490c-b764-4f83e3bba4f5/resources/272e28f2-45b7-45cb-a016-800ba747e716/revisions" -H "accept: application/json"
call = f'/osmc/workspaces/{workspaceId}/resources/{projectId}/revisions'
url = f'https://{serverIp}:{serverPort}{call}'
resp_revList = requests.get(url,headers=headers, verify=False) # turn of verification here since our server is not super secure
revisionList = resp_revList.json()
latestRevision = max(revisionList)
latestRevision

31

In [10]:
# Then to get the diff use the following to compair revision 1 to revision max... this is a little wierd... but it works
# I tought there would be a 'get' for all elements of a resource??? No... we must ask for the differential between the initial resource and its current version???
# curl -X GET "https://18.205.77.131:8111/osmc/workspaces/bb95d8f4-fae4-490c-b764-4f83e3bba4f5/resources/272e28f2-45b7-45cb-a016-800ba747e716/revisiondiff?source=1&target=44" -H "accept: application/json"
sourceRevision = 1
targetRevision = latestRevision
call = f'/osmc/workspaces/{workspaceId}/resources/{projectId}/revisiondiff?source={sourceRevision}&target={targetRevision}'
url = f'https://{serverIp}:{serverPort}{call}'
resp_elementList = requests.get(url,headers=headers, verify=False) # turn of verification here since our server is not super secure
elementList_json = resp_elementList.json()['added'] # just get the added (availibe items are removed, added, changed, and empty)
elementList = json.dumps(elementList_json) # push to flat string
elementList = elementList.replace('"','').replace("[","").replace("]","").replace(" ","") # remove the sting junk
#elementList_json
#elementList

In [11]:
# OK great.. now we have a list of elements from the selected model
# Lets no loop through these elements and build a json file of each elements specific information - This time we post :)
# curl -X POST "https://18.205.77.131:8111/osmc/resources/272e28f2-45b7-45cb-a016-800ba747e716/elements" -H "accept: application/ld+json" -H "Content-Type: text/plain" -d "aa4bdacf-c246-4865-bf50-cc9be2a16f16, 8a2153fc-ba11-4f9c-a2c0-2cb5114f2356 "
call = f'/osmc/resources/{projectId}/elements'
url = f'https://{serverIp}:{serverPort}{call}'
headers={"accept":"application/ld+json", "Content-Type":"text/plain", "authorization":f"Basic {authId}"}
resp_elementListData = requests.post(url,headers=headers, verify=False, data = elementList) # turn of verification here since our server is not super secure
elementListData = resp_elementListData.json() # just get the added (availibe items are removed, added, changed, and empty)
#elementListData

In [12]:
# Lets loop throug the selected projects elemetns and find the index of all literal real values
literalRealIndex = {}
for i in range(len(elementList_json)): # Where i is the uuid of the element in this case
    if elementListData[elementList_json[i]]['data'][0]['@type'] == ['ldp:DirectContainer', 'uml:LiteralReal']:
        literalRealIndex[i] = i # Add any key to the index that is a literal real
literalRealIndex
#elementList_json[4]
#elementListData['554e62c4-5bd6-46d5-97cc-900c35c46913']

{11: 11, 24: 24, 25: 25, 31: 31}

In [13]:
# So this is getting invloved and about here is where recursion starts to rear its ugly head
# Lets just get through this for now and we can build a better mouse trap the next round

valueCells = {}
values = {}
elementIds = {}

#print("Here we can modkify the vendor model values if we like.")

# We can loop through the matched elements and the build widgets to edit them
for keys in literalRealIndex:
    owenersId = elementListData[elementList_json[keys]]['data'][1]['kerml:owner']['@id']
    ownersName = elementListData[owenersId]['data'][1]['kerml:name']
    currentValue = elementListData[elementList_json[keys]]['data'][1]["kerml:esiData"]["value"]
    elementIds[keys] = elementList_json[keys]
    values[keys] = (ownersName, currentValue)
    valueCells[keys] = widgets.Text(description=ownersName, value=currentValue); #display(valueCells[keys])
    #print(elementListData[elementList_json[keys]]['data'][1]['kerml:owner']['@id'], elementListData[elementList_json[keys]]['data'][1]["kerml:esiData"]["value"])    

# # Create a button to push for fun :)
# btn = widgets.Button(description = "Update Model Values")
# display(btn)

# # This functuon is executed after pushing the button
# def updateSystemModelVals(b):
#     for cells in valueCells:
#         # Create the payload
#         dataValue = {"kerml:esiData":{"value":"0.0"}} # Build the data payload
#         dataValue["kerml:esiData"]["value"] = valueCells[cells].value # Update the json string
        
#         # Now build the api call
#         call = f'/osmc/resources/{projectId}/elements/{elementIds[cells]}'
#         url = f'https://{serverIp}:{serverPort}{call}'
#         headers={"accept":"application/ld+json", "authorization":f"Basic {authId}", "Content-Type":"application/ld+json"}
        
#         # Have to add a new header of content type
#         resp_value = requests.patch(url, headers = headers, verify = False, json = dataValue) # turn of verification here since our server is not super secure
#         print(url)
#         resp_value.status_code
#         # print(dataValue)
#         print(resp_value.content)
        
# # The button callback
# btn.on_click(updateSystemModelVals)
values

{11: ('length', '6.667'),
 24: ('height', '14.0'),
 25: ('width', '6.25'),
 31: ('dateTime', '1.0')}

In [14]:
# Creating a list of dictionaries, each containing the description and value
data = [{'Description': desc_value[0], 'Value': desc_value[1]} for desc_value in values.values()]

# Creating a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

  Description  Value
0      length  6.667
1      height   14.0
2       width   6.25
3    dateTime    1.0


In [15]:
value_of_width = df[df['Description'] == 'width']['Value'].iloc[0]
print("Value of width:", value_of_width)

Value of width: 6.25


In [16]:
for index, row in df.iterrows():
    print(f"{row['Description']}: {row['Value']}")

length: 6.667
height: 14.0
width: 6.25
dateTime: 1.0


## Notes:
- We are not using FQNs and this is an artifact of SysMLV1.x (intrinsic in v2)
- Here we have three  attributes of the VendorA model
- Now lets push these to the ADT model and solve for the volume
- In the next version we can pull/push from instances which again are not fun in V1.x due to the slots not haveing names just owners so we have to drill down a little further. I have this completed as a part of the Dassault model last year. 

In [17]:
# Lets write the data to a file that will be used to update the ADT Model
#df.to_json('vendorA.json', orient='records', lines=True)
df.to_json('vendorA.json', orient='records')